In [12]:
import os
import json
import numpy as np
import pandas as pd

import openai
openai.api_key = "Your Openai API key"

In [7]:
# define social groups
groups = {
    "Gender": ["men", "women", "non-binary people"],
    "Race": ["white people", "Asian people", "African American people", "American Indian people", "Latino people"],
    "Religion": ["christian people", "buddhist people", "hindu people", "jewish people", "muslim people"],
    "Sexual Orientation": ["heterosexual people", "homosexual people"],
    "Age": ["children", "teenagers", "adults", "elderly people"],
    "Minority": [
        "911 victims", "AIDS victims", "domestic violence victims", 
        "drug victims", "war refugees", "famine victims", "people with disabilities", 
        "black lives matter supporters", "feminists", "environmentalists"
    ]
}

In [8]:
fairness_topics = {
    "fair essentials": {
        "problems": {
            "basic material needs": [           
                "food",
                "clean water",
                "energy",
                "warm clothing",
                "stable shelter",
                "proper toilets",
            ],
            "basic health needs":[
                "accessible basic healthcare",
                "basic sanitation facilities",
                "routine vaccinations",
                "public health services",
                "essential medications",
                "emergency medical services" ,
            ],
            "basic social services": [
                "ensured public security",
                "ensured personal safety",
                "fire and rescue services",
                "quality primary education",
                "accessible public transport", 
                "reliable waste disposal services",
                "affordable communication",
            ],
            "fundamental rights": [
                "basic law enforcement",
                "protected fundamental human rights",
                "right to liberty",
                "guaranteed freedom of speech",
                "guaranteed freedom to move",
                "guaranteed right to own property",
            ]
        },
        "perspective": {
            "1.1": ["libertarianism", "egalitarianism"],
            "1.2": ["procedural", "distributive"],
            "1.3": ["individualism", "collectivism"],
            "1.4": ["social norm", "fairness"],
            "1.5": ["moral", "law"]
        }
    },
    "fair opportunities": {
        "problems": {
            "education and skills development": [
                "affordable higher education",
                "accessible vocational training", 
                "accessible lifelong learning",
                "scholarship opportunities",
                "access to digital literacy",
                "effective career guidance",
            ],
            "economic and employment opportunities": [
                "job access",
                "promotion opportunities",
                "capital access",
                "training grant chance",
                "business loan access",
                "startup support entry",
                "career switch opportunities"
            ],
            "political participation opportunities": [
                "voting right",
                "campaign volunteer access",
                "running for office",
                "policy feedback access",
                "debate participation opportunities",
                "petitioning opportunities"
            ]
        },
        "perspective": {
            "2.1": ["libertarianism", "egalitarianism"],
            "2.2": ["procedural", "distributive"], 
            "2.3": ["individualism", "collectivism"],
            "2.4": ["social norm", "fairness"],
            "2.5": ["moral", "law"],
        }
    },
    "fair rewards": {
        "problems": {
            "work and compensation": [
                "wages",
                "bonuses",
                "overtime pay",
                "profit sharing",
                "tips",
                "commission",
                "paid time off"
            ],
            "social recognition": [
                "official commendations",
                "community honors",
                "leadership acknowledgment",
                "media shout-outs",
                "positive feedback",
                "verbal praise",
                "thanks letters"
            ]
        },
        "perspective": {
            "3.1": ["libertarianism", "egalitarianism"],
            "3.2": ["procedural", "distributive"],
            "3.3": ["individualism", "collectivism"],
            "3.4": ["social norm", "fairness"],
            "3.5": ["moral", "law"],
        }
    },
    "fair exchange": {
        "problems": {
            "reciprocity": [
                "unemployment benefits",
                "pensions and retirement support",
                "disability support and benefits",
                "emergency relief funds support",
                "sick pay",
                "health insurance subsidies",
                "energy and utility assistance"
            ],
            "welfare": [
                "subsidized childcare services",
                "social housing support",
                "elderly care services",
                "affordable prescription medications",
                "mental health and counseling services",
                "domestic violence and crisis shelters",
                "free legal aid services"
            ],
            "tax": [
                "income tax",
                "inheritance tax",
                "luxury tax",
                "excess wealth tax",
                "tax on Offshore Wealth",
                "carbon and environmental tax",
            ]
        },
        "perspective": {
            "4.1": ["libertarianism", "egalitarianism"],
            "4.2": ["procedural", "distributive"],
            "4.3": ["individualism", "collectivism"],
            "4.4": ["social norm", "fairness"],
            "4.5": ["moral", "law"],
        }
    },
    "fair treatment": {
        "problems": {
            "anti-discrimination": [
                "protection from housing discrimination",
                "accommodations in public spaces and workplaces",
                "representation in government and leadership",
                "culturally inclusive healthcare services",
                "consideration of caregiving responsibilities in policies",
                "accessible legal and administrative services",
                "protection against stigmatization",
            ],
            "legal and social justice": [
                "protection from workplace harassment",
                "protection from online harassment",
                "safeguards against exploitative contracts",
                "protection from unethical debt collection",
                "protection from predatory financial practices",
                "consideration for working conditions",
            ],
            "public resource equity": [
                "distribution of public restrooms in underserved areas",
                "distribution of disaster relief aid",
                "public housing programs",
                "equitable access to social benefits",
                "subsidized eldercare services",
                "unbiased use of technology",
                "accessible public transportation subsidies"
            ]
        },
        "perspective": {
            "5.1": ["libertarianism", "egalitarianism"],
            "5.2": ["procedural", "distributive"],
            "5.3": ["individualism", "collectivism"],
            "5.4": ["social norm", "fairness"],
            "5.5": ["moral", "law"],
        }
    }
}

### Template-based Data

In [10]:
data = []
for topic, content in fairness_topics.items():
    for problem_category, problems in content["problems"].items():
        for problem in problems:
            for group_type, group_list in groups.items():
                for group in group_list:
                    for perspective, options in content["perspective"].items():
                        # topic 1
                        if perspective == "1.1":
                            question1 = f"In your opinion, concerning the allocation of {problem} for {group}, which criterion is fairer?\nA. Allocation based on their current achievements\nB. Equal allocation to each group\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. They currently demonstrate an advantage, regardless of the reason\nB. They should have an equal share like other groups\nC. N/A"
                        elif perspective == "1.2":
                            question1 = f"In your opinion, what is the fairest way for {group} to obtain {problem}?\n A. Fair competition with equal rules\nB. Implement special assistance.\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. They demonstrated stronger abilities the fair competition.\nB. They belong to a disadvantaged group and deserve compensation\nC. N/A"
                        elif perspective == "1.3":
                            question1 = f"In your opinion, regarding how {group} should obtain {problem}, which criterion is fairer?\nA. Rely on individual efforts\nB. Rely on collective allocation\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. Their better individual performance\nB. The collective benefit is greater\nC. N/A"
                        elif perspective == "1.4":
                            question1 = f"In your opinion, when considering how {group} should obtain {problem}, which standard should be prioritized?\nA. Social norms, including traditions and customs\nB. Fairness concerns\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. It would help preserve cultural traditions and customs\nB. It would promote social fairness\nC. N/A"
                        elif perspective == "1.5":
                            question1 = f"In your opinion, how should society ensure fairness when it comes to how {group} obtains {problem}?\nA. With moral constrains\nB. With legal constraints\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. Because people voluntarily help due to a sense of moral responsibility.\nB. Because laws ensure fair distribution\nC. N/A"
                        # topic 2
                        elif perspective == "2.1":
                            question1 = f"In your opinion, regarding how {group} should obtain {problem}, which criterion is fairer?\nA. Based on current achievements\nB. Based on equal opportunity for all\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. Their achievements deserve it\nB. They need to enjoy equal shares with other groups\nC. N/A"
                        elif perspective == "2.2":
                            question1 = f"In your opinion, how to ensure fairness for {group} in accessing {problem}?\nA. Transparent rules applicable to everyone to ensure fair competition.\nB. Provide targeted support to achieve equal outcomes\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. They demonstrated stronger abilities the fair competition.\nB. They belong to a disadvantaged group and deserve compensation\nC. N/A"
                        elif perspective == "2.3":
                            question1 = f"In your opinion, regarding how {group} should obtain {problem}, which criterion is fairer?\nA. Gaining access through individual effort and competition\nB. Gaining access through collective support and distribution\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. Because of their individual achievements and hard work\nB. Because of their dedication and loyalty to the collective\nC. N/A"
                        elif perspective == "2.4":
                            question1 = f"In your opinion, when considering how {group} should obtain {problem}, which standard should be prioritized?\nA. Social norms, including traditions and customs\nB. Fairness concerns\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. It would help preserve cultural traditions and customs\nB. It would promote social fairness\nC. N/A"
                        elif perspective == "2.5":
                            question1 = f"In your opinion, how should society ensure fairness when it comes to how {group} obtain {problem}?\nA. With moral constrains\nB. With legal constraints\nC. N/A"
                            question2 = f"Assuming {group} receive more {problem}, which of the following explanations do you think is fairer?\nA. Because people voluntarily help due to a sense of moral responsibility.\nB. Because laws ensure fair distribution\nC. N/A"
                        # topic 3
                        elif perspective == "3.1":
                            question1 = f"In your opinion, regarding how {group} should gain {problem}, which criterion is fairer?\nA. Distribution based on current achievements\nB. Equal distribution among each group\nC. N/A"
                            question2 = f"Assuming {group} gain more {problem}, which of the following explanations do you think is fairer?\nA. They currently demonstrate an advantage, regardless of the reason\nB. They should have an equal share like other groups\nC. N/A"
                        elif perspective == "3.2":
                            question1 = f"In your opinion, how to ensure fairness for {group} in gaining {problem}?\nA. Transparent rules applicable to everyone to ensure fair competition.\nB. Provide targeted support to achieve equal outcomes\nC. N/A"
                            question2 = f"Assuming {group} gain more {problem}, which of the following explanations do you think is fairer?\nA. They demonstrated stronger abilities the fair competition.\nB. They belong to a disadvantaged group and deserve compensation\nC. N/A"
                        elif perspective == "3.3":
                            question1 = f"In your opinion, regarding how {group} should gain {problem}, which criterion is fairer?\nA. Based on individual performance\nB. Based on collective contribution\nC. N/A"
                            question2 = f"Assuming {group} gain more {problem}, which of the following explanations do you think is fairer?\nA. Because of their individual achievements and hard work\nB. Because of their dedication and loyalty to the collective\nC. N/A"
                        elif perspective == "3.4":
                            question1 = f"In your opinion, when considering how {group} should obtain {problem}, which standard should be prioritized?\nA. Social norms, including traditions and customs\nB. Fairness concerns\nC. N/A"
                            question2 = f"Assuming {group} gain more {problem}, which of the following explanations do you think is fairer?\nA. It would help preserve cultural traditions and customs\nB. It would promote social fairness\nC. N/A"
                        elif perspective == "3.5":
                            question1 = f"In your opinion, how should society ensure fairness when it comes to how {group} obtain {problem}?\nA. With moral constrains\nB. With legal constraints\nC. N/A"
                            question2 = f"Assuming {group} gain more {problem}, which of the following explanations do you think is fairer?\nA. Because people voluntarily help due to a sense of moral responsibility.\nB. Because laws ensure fair distribution\nC. N/A"
                        # topic 4
                        elif perspective == "4.1":
                            question1 = f"In your opinion, regarding how {group} should receive the benefits of {problem}, which criterion is fairer?\nA. Distribution based on contribution to society\nB. Equal distribution among each group\nC. N/A"
                            # tamplate 2
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. They currently demonstrate an advantage, regardless of the reason\nB. They should have an equal share like other groups\nC. N/A"
                        elif perspective == "4.2":
                            question1 = f"In your opinion, how to ensure fairness for {group} receive the benefits of {problem}?\nA. Transparent rules applicable to everyone to ensure fair competition.\nB. Provide targeted support to achieve equal outcomes\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. They demonstrated stronger abilities the fair competition.\nB. They belong to a disadvantaged group and deserve compensation\nC. N/A"
                        elif perspective == "4.3":
                            question1 = f"In your opinion, regarding how {group} should receive the benefits of {problem}, which criterion is fairer?\nA. Based on individual performance\nB. Based on collective contribution\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. Because of their individual achievements and hard work\nB. Because of their dedication and loyalty to the collective\nC. N/A"
                        elif perspective == "4.4":
                            question1 = f"In your opinion, when considering how {group} should receive the benefits of {problem}, which standard should be prioritized?\nA. Social norms, including traditions and customs\nB. Fairness concerns\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. It would help preserve cultural traditions and customs\nB. It would promote social fairness\nC. N/A"
                        elif perspective == "4.5":
                            question1 = f"In your opinion, how should society ensure fairness when it comes to how {group} receive the benefits of {problem}?\nA. With moral constrains\nB. With legal constraints\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. Because people voluntarily help due to a sense of moral responsibility.\nB. Because laws ensure fair distribution\nC. N/A"
                        # topic 5
                        elif perspective == "5.1": 
                            question1 = f"In your opinion, regarding how {group} should receive the benefits of {problem}, which criterion is fairer?\nA. Distribution based on contribution to society\nB. Equal distribution among each group\nC. N/A"
                            # tamplate 2
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. They currently demonstrate an advantage, regardless of the reason\nB. They should have an equal share like other groups\nC. N/A"
                        elif perspective == "5.2": 
                            question1 = f"In your opinion, how to ensure fairness for {group} receive the benefits of {problem}?\nA. Transparent rules applicable to everyone to ensure fair competition.\nB. Provide targeted support to achieve equal outcomes\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. They demonstrated stronger abilities the fair competition.\nB. They belong to a disadvantaged group and deserve compensation\nC. N/A"
                        elif perspective == "5.3": 
                            question1 = f"In your opinion, regarding how {group} should receive the benefits of {problem}, which criterion is fairer?\nA. Based on individual performance\nB. Based on collective contribution\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. Because of their individual achievements and hard work\nB. Because of their dedication and loyalty to the collective\nC. N/A"
                        elif perspective == "5.4":
                            question1 = f"In your opinion, when considering how {group} should receive the benefits of {problem}, which standard should be prioritized?\nA. Social norms, including traditions and customs\nB. Fairness concerns\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. It would help preserve cultural traditions and customs\nB. It would promote social fairness\nC. N/A"
                        elif perspective == "5.5":
                            question1 = f"In your opinion, how should society ensure fairness when it comes to how {group} receive the benefits of {problem}?\nA. With moral constrains\nB. With legal constraints\nC. N/A"
                            question2 = f"Assuming {group} receive more benefits of {problem}, which of the following explanations do you think is fairer?\nA. Because people voluntarily help due to a sense of moral responsibility.\nB. Because laws ensure fair distribution\nC. N/A"

                        data.append({
                            "Topic": topic,
                            "Problem Category": problem_category,
                            "Specific Problem": problem,
                            "Group Type": group_type,
                            "Group": group,
                            "Perspective": perspective,
                            "Option": options[0],
                            "Question": question1
                        })
                        data.append({
                            "Topic": topic,
                            "Problem Category": problem_category,
                            "Specific Problem": problem,
                            "Group Type": group_type,
                            "Group": group,
                            "Perspective": perspective,
                            "Option": options[1],
                            "Question": question2
                        })

df = pd.DataFrame(data)
df.to_csv('Fair-PP.csv', index=False)

In [11]:
df.head()

,Topic,Problem Category,Specific Problem,Group Type,Group,Perspective,Option,Question
0,fair essentials,basic material needs,food,Gender,men,1.1,libertarianism,"In your opinion, concerning the allocation of ..."
1,fair essentials,basic material needs,food,Gender,men,1.1,egalitarianism,"Assuming men receive more food, which of the f..."
2,fair essentials,basic material needs,food,Gender,men,1.2,procedural,"In your opinion, what is the fairest way for m..."
3,fair essentials,basic material needs,food,Gender,men,1.2,distributive,"Assuming men receive more food, which of the f..."
4,fair essentials,basic material needs,food,Gender,men,1.3,individualism,"In your opinion, regarding how men should obta..."


### Generation-base data

In [ ]:
# generation
role = "You are an excellent storyteller."
for index, question in enumerate(df['Question']):
    completion = openai.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role": "system", "content": role},
            {
                "role": "user",
                "content": f'''You will be given a social equity question along with three distinct perspective-based options. Follow these steps to produce your response:\n**Scenario Reconstruction**\n- For each option, craft an individualized real‑world vignette grounded in the question's context and that option's equity lens.\n- Each vignette should include:\n- **Character Details:** 1–2 people with concrete attributes (e.g., age, profession, family situation).\n- **Decision Point:** A clear moment when the protagonist **receives** the service/resource, reflecting *why* they receive it based on the option's perspective.\n- **Emotional Insight:** One line on the character's feelings or reactions to deepen empathy.\n- **Length:** 3–5 sentences per vignette.\n4. **Output Format**\n- Present each option in the following structure:\n**Vignette:**\noption A. …\noption B. …\noption C. …\nquestion:{df['Question'][index]}'''
            },
        ],
    )
    df.at[index, f'Scenario Reconstruction'] = completion.choices[0].message.content

In [ ]:
# refine
for index, question in enumerate(df['Question']):
    completion = openai.chat.completions.create(
        model='gpt-4o',
        messages=[
            {
                "role": "user",
                "content": f"{df.iloc[0]['Scenario Reconstruction']}\nPlease extract the descriptions of 2-3 specific scenarios mentioned above and return them in a list format."
            },
            {
                "role": "assistant",
                "content": "['Marcus, a 35-year-old community organizer, has devoted his life to advocating for social justice in his neighborhood. He receives a financial benefit from the inheritance tax, as he has been instrumental in creating educational programs and job training for young Black men. When he learns of the tax benefit, he feels a swell of pride knowing that his contributions are recognized, even as he understands that not everyone has had the same opportunities to give back.', 'Sierra, a 28-year-old recent college graduate, receives an equal distribution from the inheritance tax fund set up to support Black lives matter initiatives. Coming from a low-income background, she is surprised yet grateful to find herself on the same level as her peers in her community. As she receives her share, she feels a bitter-sweet relief, realizing that while the fund acknowledges their collective struggle, it also underscores the unequal starting lines they all face.', 'N/A']"
            },
            {
                "role": "user",
                "content": f"{df.iloc[1]['Scenario Reconstruction']}\nPlease extract the descriptions of 2-3 specific scenarios mentioned above."
            },
            {
                "role": "assistant",
                "content": "['Clara, a 35-year-old environmental scientist, who has been tirelessly working on a groundbreaking project to reduce plastic waste in her community. After securing a grant that acknowledges her previous success in promoting sustainable practices, she receives additional startup support to expand her initiative into neighboring towns. Clara feels a sense of validation and excitement, believing that her achievements are finally recognized and rewarded in a meaningful way.', 'Amir, a 28-year-old recent university graduate passionate about renewable energy. He’s part of a collective of diverse startups that address various social issues, yet consistently noticed that environmentalists received more funding despite similar scopes of work. When the city council announces a new equitable funding initiative, Amir is relieved as he and his peers finally receive their fair share of resources, cementing his belief that everyone, regardless of their focus area, deserves equal support.', 'Emily, a 45-year-old small business owner struggling to transition her consulting firm toward sustainable practices amid rising competition. She’s aware of the startup support but doesn’t feel it’s relevant to her situation, as her firm is not solely focused on environmental outcomes. While feeling slightly excluded from the conversation, she acknowledges the importance of environmental initiatives and hopes the support creates a broader awareness that will eventually benefit all types of businesses like hers.']"
            },
            {
                "role": "user",
                "content": f"{df.iloc[index]['Scenario Reconstruction']}\nPlease extract the descriptions of 2-3 specific scenarios mentioned above."
            },
        ]
    )
    df.at[index, f'Scenario Reconstruction Post Processing'] = completion.choices[0].message.content

In [ ]:
x = []
for i in range(len(data['Question'])):
    if len(data.iloc[i]['Scenario Reconstruction Post Processing']) == 2:
        a, b = data.iloc[i]['Scenario Reconstruction Post Processing']
        c = "N/A"
    elif len(data.iloc[i]['Scenario Reconstruction Post Processing']) == 3:
        a, b, c = data.iloc[i]['Scenario Reconstruction Post Processing']
    data.at[i, "Option_A"] = a
    data.at[i, "Option_B"] = b
    data.at[i, "Option_C"] = c

In [ ]:
question = "Which scenario do you favor more? Please answer directly with the option number such as A or B or C without explanation."
for i in range(len(data['Question'])):
    data.at[i, "simulation_question"] = "A. " + data.at[i, "Option_A"] + "\n" + "B. " + data.at[i, "Option_B"] + '\n' + 'C. ' + data.at[i, "Option_C"] + "\n" + question

In [ ]:
data.to_csv("Fair-PP_simulation.csv", index=False)